In [1]:
import os
import numba
%load_ext Cython

In [2]:
os.chdir("/Users/kei/Projects/spykesim/drafts/")

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import line_profiler
import cython
%matplotlib inline
%load_ext line_profiler

In [6]:
# Row: Neuron #, Col: Bin #
mat1 = np.array([
    [1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]])
mat2 = np.array([
    [1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1],
    [1, 1, 1, 1, 0],
    [1, 1, 1, 0, 0],
    [1, 0, 0, 0, 0]])

# Simple editsim

In [7]:
def simpleeditsim(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            dp_table[col1 + 1, col2 + 1] = np.max([
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match])
    return dp_table[-1, -1]


In [32]:
%%cython -a
import numpy as np
cimport cython
cimport numpy as np
DBL = np.double
ctypedef np.double_t DBL_C


cdef inline float float_min2(float a, float b):
    return a if a <= b else b
cdef inline float float_min(float a, float b, float c):
    cdef float d = float_min2(a, b)
    cdef float e = float_min2(b, c)
    return float_min2(d, e)
cdef inline float float_max2(float a, float b):
    return a if a >= b else b
cdef inline float float_max(float a, float b, float c):
    cdef float d = float_max2(a, b)
    cdef float e = float_max2(b, c)
    return float_max2(d, e)
cdef inline int int_max(int a, int b):
    return a if a >= b else b
cdef inline int int_min(int a, int b):
    return a if a <= b else b

In [37]:
%%cython -a
import numpy as np
cimport cython
cimport numpy as np
DBL = np.double
ctypedef np.double_t DBL_C


cdef inline float float_min2(float a, float b):
    return a if a <= b else b
cdef inline float float_min(float a, float b, float c):
    cdef float d = float_min2(a, b)
    cdef float e = float_min2(b, c)
    return float_min2(d, e)
cdef inline float float_max2(float a, float b):
    return a if a >= b else b
cdef inline float float_max(float a, float b, float c):
    cdef float d = float_max2(a, b)
    cdef float e = float_max2(b, c)
    return float_max2(d, e)
cdef inline int int_max(int a, int b):
    return a if a >= b else b
cdef inline int int_min(int a, int b):
    return a if a <= b else b
        
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(True)  # turn off negative index wrapping for entire function
def csimpleeditsim(DBL_C [:, :] mat1, DBL_C [:, :] mat2):
    cdef int nrow, ncol, nneuron
    cdef int col1, col2, row
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    nneuron = mat1.shape[0]
    cdef np.ndarray[DBL_C, ndim=2] dp_table = np.zeros((nrow+1, ncol+1), dtype = DBL)
    cdef DBL_C match
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = 0
            for row in range(nneuron):
                match += mat1[row, col1] * mat2[row, col2]
            dp_table[col1 + 1, col2 + 1] = float_max(
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match)
    return dp_table[-1, -1]

In [ ]:
%%cython -a
import numpy as np
cimport cython
cimport numpy as np
DBL = np.double
ctypedef np.double_t DBL_C


cdef inline float float_min2(float a, float b):
    return a if a <= b else b
cdef inline float float_min(float a, float b, float c):
    cdef float d = float_min2(a, b)
    cdef float e = float_min2(b, c)
    return float_min2(d, e)
cdef inline float float_max2(float a, float b):
    return a if a >= b else b
cdef inline float float_max(float a, float b, float c):
    cdef float d = float_max2(a, b)
    cdef float e = float_max2(b, c)
    return float_max2(d, e)
cdef inline int int_max(int a, int b):
    return a if a >= b else b
cdef inline int int_min(int a, int b):
    return a if a <= b else b
        
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(True)  # turn off negative index wrapping for entire function
def csimpleeditsim(DBL_C [:, :] mat1, DBL_C [:, :] mat2):
    cdef int nrow, ncol, nneuron
    cdef int col1, col2, row
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    nneuron = mat1.shape[0]
    cdef np.ndarray[DBL_C, ndim=2] dp_table = np.zeros((nrow+1, ncol+1), dtype = DBL)
    cdef DBL_C match
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = 0
            for row in range(nneuron):
                match += mat1[row, col1] * mat2[row, col2]
            dp_table[col1 + 1, col2 + 1] = float_max(
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match)
    return dp_table[-1, -1]

In [3]:
nrow = 100
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)

In [43]:
import sys

In [47]:
sys.path.append("build/lib.macosx-10.9-x86_64-3.6")

In [4]:
import editsim

In [5]:
%timeit editsim.csimpleeditsim(mat1, mat2)

910 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%timeit csimpleeditsim(mat1, mat2)

896 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
%timeit csimpleeditsim(mat1, mat2)

902 µs ± 8.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%timeit csimpleeditsim(mat1, mat2)

896 µs ± 19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%timeit csimpleeditsim(mat1, mat2)

887 µs ± 5.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%timeit csimpleeditsim(mat1, mat2)

7.22 ms ± 52.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit csimpleeditsim(mat1, mat2)

3.68 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit simpleeditsim(mat1, mat2)

68.2 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Simple editsim with bp

In [164]:
def simpleeditsim_withbp(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)            
    return dp_table[-1, -1], bp_table


In [171]:
%%cython -a
import numpy as np
cimport cython
cimport numpy as np
DBL = np.double
cdef struct Pair:
    float value
    int idx
cdef inline Pair indmax(float a, float b, float c):
    cdef Pair pair
    if a >= b:
        if a >= c:
            pair.value = a
            pair.idx = 0
            return pair
        else:
            # b <= a <= c
            pair.value = c
            pair.idx = 2
            return pair
    else:
        # a <= b
        if b >= c:
            # a < b and b > c
            pair.value = b
            pair.idx = 1
            return pair
        else:
            # a < b and b < c
            pair.value = c
            pair.idx = 2
            return pair

def test(a, b, c):
    print(indmax(a, b, c))

In [166]:
%%cython -a
import numpy as np
cimport cython
cimport numpy as np
DBL = np.double
ctypedef np.double_t DBL_C


cdef inline float float_min2(float a, float b):
    return a if a <= b else b
cdef inline float float_min(float a, float b, float c):
    cdef float d = float_min2(a, b)
    cdef float e = float_min2(b, c)
    return float_min2(d, e)
cdef inline float float_max2(float a, float b):
    return a if a >= b else b
cdef inline float float_max(float a, float b, float c):
    cdef float d = float_max2(a, b)
    cdef float e = float_max2(b, c)    
    return float_max2(d, e)
cdef struct Pair:
    float value
    int idx
cdef inline Pair indmax(float a, float b, float c):
    cdef Pair pair
    if a >= b:
        if a >= c:
            pair.value = a
            pair.idx = 0
            return pair
        else:
            # b <= a <= c
            pair.value = c
            pair.idx = 2
            return pair
    else:
        # a <= b
        if b >= c:
            # a < b and b > c
            pair.value = b
            pair.idx = 1
            return pair
        else:
            # a < b and b < c
            pair.value = c
            pair.idx = 2
            return pair
cdef inline 
cdef inline int int_max(int a, int b):
    return a if a >= b else b
cdef inline int int_min(int a, int b):
    return a if a <= b else b


def simpleeditsim_withbp(DBL_C [:, :] mat1, DBL_C [:, :] mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)            
    return dp_table[-1, -1], bp_table

def csimpleeditsim(DBL_C [:, :] mat1, DBL_C [:, :] mat2):
    cdef int nrow, ncol, nneuron
    cdef int col1, col2, row
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    nneuron = mat1.shape[0]
#     dp_table_np = np.zeros((nrow+1, ncol+1), dtype = DBL)
#     cdef DBL_C [:,:] dp_table = dp_table_np
    cdef np.ndarray[DBL_C, ndim=2] dp_table = np.zeros((nrow+1, ncol+1), dtype = DBL)
    cdef float match
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = 0
            for row in range(nneuron):
                match += mat1[row, col1] * mat2[row, col2]
            dp_table[col1 + 1, col2 + 1] = float_max(
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match)
    return dp_table[-1, -1]

In [165]:
simpleeditsim_withbp(mat1, mat2)

(200492.0, array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1.,  2.,  2., ...,  1.,  1.,  1.],
        [-1.,  0.,  2., ...,  1.,  1.,  1.],
        ..., 
        [-1.,  0.,  0., ...,  2.,  2.,  2.],
        [-1.,  0.,  0., ...,  2.,  2.,  2.],
        [-1.,  0.,  0., ...,  2.,  2.,  2.]]))

In [163]:
print(simpleeditsim(mat1, mat2), csimpleeditsim(mat1, mat2))

200492.0 200492.0


In [38]:
%%cython -a
import numpy as np
class Editsim(object):
    def __init__(self, method = "simple", with_bp = True, alpha = 0.01):
        self.method = method
        self.with_bp = with_bp
        self.alpha = alpha
        if (method == "simple") and not with_bp:
            self.editsim = self._simpleeditsim
        elif (method == "simple") and with_bp:
            self.editsim = self._simpleeditsim_withbp
            self.align = self._simpleeditsim_align
    def _simpleeditsim(self, mat1, mat2):
        nrow = mat1.shape[1]
        ncol = mat2.shape[1]
        dp_table = np.zeros((nrow+1, ncol+1))
        for col1 in range(nrow):
            for col2 in range(ncol):
                match = np.dot(mat1[:, col1], mat2[:, col2])
                dp_table[col1 + 1, col2 + 1] = np.max([
                    dp_table[col1, col2 + 1],
                    dp_table[col1 + 1, col2],
                    dp_table[col1, col2] + match])
        return dp_table[-1, -1]
    def _simpleeditsim_withbp(self, mat1, mat2):
        nrow = mat1.shape[1]
        ncol = mat2.shape[1]
        dp_table = np.zeros((nrow+1, ncol+1))
        bp_table = np.ones_like(dp_table) * (-1)
        for col1 in range(nrow):
            for col2 in range(ncol):
                match = np.dot(mat1[:, col1], mat2[:, col2])
                choices = [
                    dp_table[col1, col2 + 1],
                    dp_table[col1 + 1, col2],
                    dp_table[col1, col2] + match
                ]
                bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
                dp_table[col1 + 1, col2 + 1] = np.max(choices)            
        return dp_table[-1, -1], bp_table
    def _simpleeditsim_align(self, bp, mat1, mat2):
        nrow = bp.shape[0]
        ncol = bp.shape[1]
        row = nrow - 1
        col = ncol - 1
        # The first column is inserted just to avoid initialization error that may occur on concatination.
        alignment1 = np.zeros((mat1.shape[0], 1))
        alignment2 = np.zeros((mat1.shape[0], 1))
        zerovec = np.zeros(mat1.shape[0]) # which is corresponding to the null character.
        while True:
            if bp[row, col] == -1:
                # Eather of the strings tracing terminated
                break
            elif bp[row, col] == 2:
                alignment1 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment1]
                alignment2 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment2]
                row -= 1
                col -= 1
            elif bp[row, col] == 1:
                alignment1 = np.c_[zerovec, alignment1]
                alignment2 = np.c_[mat2[:, col - 1], alignment2]
                col -= 1
            elif bp[row, col] == 0:
                alignment1 = np.c_[mat1[:, row - 1], alignment1]
                alignment2 = np.c_[zerovec, alignment2]
                row -= 1
        return alignment1[:, :-1], alignment2[:, :-1]

In [41]:
editsim = Editsim(with_bp = True)

In [42]:
sim, bp = editsim.editsim(mat1, mat2)

In [29]:
editsim.align(bp, mat1, mat2)

(array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 1.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  1.,  1.,  0.],
        [ 1.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]))

In [43]:
nrow = 100
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)

In [34]:
%timeit editsim.editsim(mat1, mat2)

235 ms ± 984 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%timeit editsim.editsim(mat1, mat2)

243 ms ± 7.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
import numpy as np
def editsim(mat1, mat2, method = "simple", with_bp = False):
    return _simpleeditsim(mat1, mat2)
def trace(bp, mat1, mat2):
    pass
def _simpleeditsim(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            dp_table[col1 + 1, col2 + 1] = np.max([
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match])
    return dp_table[-1, -1]

def _simpleeditsim_withbp(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)            
    return dp_table, bp_table

def _simpleeditsim_tracebp(bp, mat1, mat2):
    nrow = bp.shape[0]
    ncol = bp.shape[1]
    row = nrow - 1
    col = ncol - 1
    # The first column is inserted just to avoid initialization error that may occur on concatination.
    alignment1 = np.zeros((mat1.shape[0], 1))
    alignment2 = np.zeros((mat1.shape[0], 1))
    zerovec = np.zeros(mat1.shape[0]) # which is corresponding to the null character.
    while True:
        if bp[row, col] == -1:
            # Eather of the strings tracing terminated
            break
        elif bp[row, col] == 2:
            alignment1 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment1]
            alignment2 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment2]
            row -= 1
            col -= 1
        elif bp[row, col] == 1:
            alignment1 = np.c_[zerovec, alignment1]
            alignment2 = np.c_[mat2[:, col - 1], alignment2]
            col -= 1
        elif bp[row, col] == 0:
            alignment1 = np.c_[mat1[:, row - 1], alignment1]
            alignment2 = np.c_[zerovec, alignment2]
            row -= 1
    return alignment1[:, :-1], alignment2[:, :-1]

def profile_simpleeditsim():
    nrow = 100
    mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    for i in range(10):
        a = _simpleeditsim(mat1, mat2)
        print(a)

In [33]:
%%timeit
profile_simpleeditsim

22.5 ns ± 0.166 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## 動作テスト

In [6]:
# A: 1-th col, T: 0-th col
# ATATATAT and
#  TATA AT
mat1 = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0]
])

mat2 = np.array([
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
])

In [9]:
dp, bp = _simpleeditsim_withbp(mat1, mat2)
print(bp)
_simpleeditsim_tracebp(bp, mat1, mat2)

NameError: name '_simpleeditsim_withbp' is not defined

In [141]:
# ATATATAT and
#  TATA AT
mat1 = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0]
])

mat2 = np.array([
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
])

# Profiling

In [10]:
nrow = 10
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)

In [12]:
%%writefile profile_editsim_draft.py
import numpy as np
from editsim_draft import *
def profile():
    nrow = 10
    mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    for i in range(10):
        a = editsim(mat1, mat2)

Overwriting profile_editsim_draft.py


In [28]:
import sys
import editsim_draft
from pathlib import Path
%lprun -T report.log -f editsim_draft._simpleeditsim editsim_draft.profile()
with Path("report.log").open( ) as rf: print(rf.readline())


*** Profile printout saved to text file 'report.log'. 
Timer unit: 1e-06 s



Timer unit: 1e-06 s

Total time: 0.036256 s
File: /Users/kei/Projects/spykesim/drafts/editsim_draft.py
Function: _simpleeditsim at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def _simpleeditsim(mat1, mat2):
     5        10         12.0      1.2      0.0      nrow = mat1.shape[1]
     6        10          7.0      0.7      0.0      ncol = mat2.shape[1]
     7        10         30.0      3.0      0.1      dp_table = np.zeros((nrow+1, ncol+1))
     8       110         60.0      0.5      0.2      for col1 in range(nrow):
     9      1100       1028.0      0.9      2.8          for col2 in range(ncol):
    10      1000       9366.0      9.4     25.8              match = np.dot(mat1[:, col1], mat2[:, col2])
    11      1000        943.0      0.9      2.6              dp_table[col1 + 1, col2 + 1] = np.max([
    12      1000        982.0      1.0      2.7                  dp_table[col1, col2 + 1],
    13      1000    

In [23]:
editsim_draft.profile()

In [ ]:
def ceval_edit_dist(fragment1, fragment2, dp_table, jitter, score_vec):
    return ceval_edit_dist_virtual(fragment1.astype(np.int32), fragment2.astype(np.int32), dp_table.astype(np.float32), jitter, 
                                 score_vec.astype(np.int32))
def ceval_edit_dist_virtual(np.ndarray[np.int32_t, ndim=2] mat1, np.ndarray[np.int32_t, ndim=2] mat2, np.ndarray[np.float32_t, ndim=2] dp_table, int jitter, 
                   np.ndarray[np.int32_t, ndim=1] score_vec):
    cdef int nrow = mat1.shape[0]
    cdef int ncol = mat1.shape[1]
    cdef int col1, col2
    cdef int row
    cdef float match=0
    cdef float disim
    for col1 in range(ncol):
        for col2 in range(int_max(0, col1-jitter), int_min(col1+jitter, ncol)):
            match = 0
            # calculate inner product
            for row in range(nrow):
                match += mat1[row, col1] * mat2[row, col2]
                match /= score_vec[row] + 1
            disim = nrow -  match
            # idx 0 : ←, 1 : ↑, 2, ↖
            dp_table[col1+1, col2+1] = float_min(dp_table[col1][col2+1]+nrow, dp_table[col1+1][col2]+nrow, dp_table[col1][col2]+disim)
    return dp_table[-1, -1]